# Geospatial analysis and visualization with PyGMT

Here, we'll demonstrate a few new (relative to SciPy 2018) features for data processing and visualization in PyGMT.

## Data visualization

Data visualization with PyGMT is handled through the [pygmt.Figure class and its methods](https://www.pygmt.org/latest/api/index.html#plotting). Examples include:

- Project and plot grids or images
- Create 3-D perspective plots
- Plot velocity vectors, crosses, and anisotropy bars
- Plot earthquake focal mechanisms


In the data visualization examples, we'll use [PyGMT](https://pygmt.org) for visualization and [geopandas](https://github.com/geopandas/geopandas) for reading and manipulating vector data. PyGMT has a flat package layout, meaning that you can access everything in it with a single `import`.

In [ ]:
import pygmt
import geopandas as gpd

### Plot earthquake focal mechanisms

First, we'll demonstrate how PyGMT provides access to the GMT's geophysical data visualization tools based on an example from the [UNAVCO-sponsored GMT for Geodesy course](https://github.com/GenericMappingTools/gmt-for-geodesy) using data from the [Global Centroid-Moment-Tensor Project](https://www.globalcmt.org/) (Ekström et al., 2012).

GMT provides easy access to several [datasets](https://www.generic-mapping-tools.org/remote-datasets). In this example, we'll load the SRTM15+ earth relief data (Tozer et al., 2019) as an xarray DataArray for the underlying map.

In [ ]:
grid = pygmt.datasets.load_earth_relief(
    resolution="02m", region=[122, 147, 30, 48], registration="gridline"
)
grid

All plotting is handled by the `pygmt.Figure` class. Here is a good analogy for it:

> `pygmt.Figure` is a blank canvas onto which you can lay down plot elements in order.

Here is how you can create a figure:

In [ ]:
fig = pygmt.Figure()

We can plot the earth relief data using the `grdimage` method of `pygmt.Figure`

In [ ]:
fig.grdimage(
    grid=grid, shading=True, projection="M15c", frame=["WSen", "af"], cmap="SCM/oleron"
)

We can add a colorbar for the earth relief data

In [ ]:
fig.colorbar(
    position="jTR+o-2c/0c+w6c+ml", frame='a2000+l"Earth relief (m)"', shading=True
)

To see what the figure looks like, we call the `show` method of `pygmt.Figure`

In [ ]:
fig.show()

Now, we can update the colormap for the earthquake focal mechanisms

In [ ]:
pygmt.makecpt(cmap="plasma", series=[0, 700, 5], reverse=True)

We can plot the focal mechanisms from the global CMT catalog using the `meca` method

In [ ]:
fig.meca("_data/japan-focal.dat", convention="mt", scale="0.5c+f0p", C=True)

Lastly, let's add another colorbar for these data and display the final result

In [ ]:
fig.colorbar(position="jBR+o-2c/0c+w-6c+ml", frame='a100+l"Focal depth (km)"')
fig.show()

We can also save the figure using the `savefig` method

In [ ]:
fig.savefig("earthquakes.png")

### Plot geospatial vector data

One of the main benefits of using PyGMT is the integration with the broader scientific Python ecosystem. In this section, we'll show how PyGMT can be used to visualize geopandas data objects based on one of the [PyGMT gallery examples](https://www.pygmt.org/latest/gallery/lines/roads.html#sphx-glr-gallery-lines-roads-py).


Read shapefile using geopandas

In [ ]:
gdf = gpd.read_file(
    "https://www2.census.gov/geo/tiger/TIGER2015/PRISECROADS/tl_2015_15_prisecroads.zip"
)

Select a few road types to plot

In [ ]:
roads_common = gdf[gdf.RTTYP == "M"]  # Common name roads
roads_state = gdf[gdf.RTTYP == "S"]  # State recognized roads
roads_interstate = gdf[gdf.RTTYP == "I"]  # Interstate roads

We'll want to define the region and title for our plot

In [ ]:
region = [-158.3, -157.6, 21.2, 21.75]
title = r"Main roads of O\047ahu (Hawai\047i)"  # \047 is octal code for '

Again, we'll use the plotting methods of `pygmt.Figure`. We'll configure the map appearance using the `basemap` method

In [ ]:
fig = pygmt.Figure()
fig.basemap(region=region, projection="M12c", frame=["af", f'WSne+t"{title}"'])

We can add some context to our map by plotting coastlines, land color, and ocean color

In [ ]:
fig.coast(land="gray", water="dodgerblue4", shorelines="1p,black")

We can easily plot the geopandas GeoDataFrames by passing them as the argument to the `data` parameter. We'll also use the `label` parameter to define the legend item for that data.

In [ ]:
fig.plot(data=roads_common, pen="5p,dodgerblue", label="Common Name")
fig.plot(data=roads_state, pen="2p,gold", label="State Recognized")
fig.plot(data=roads_interstate, pen="2p,red", label="Interstate")

Lastly, we'll add a legend and display the figure

In [ ]:
fig.legend()
fig.show()

## Processing tables | grids

PyGMT's numerous functions for processing tabular or gridded data are detailed in the [API documentation](https://www.pygmt.org/latest/api/index.html#data-processing). Examples include:
- Grid arbitrarily spaced data
- Project data onto lines or great circles
- Clip, cut, fill, and sample grids
- Filter grids in the space of time domains

### Grid LiDAR data

In this example, we'll cover a simplified version of the 2022 EGU Short Course tutorial [LiDAR point clouds to 3D surfaces](https://www.generic-mapping-tools.org/egu22pygmt/lidar_to_surface.html)!

Besides [pygmt](https://www.pygmt.org), we'll also be using:

- [laspy](https://github.com/laspy/laspy) to read in LAZ LiDAR files
- [pandas](https://pandas.pydata.org) for managing tabular data

In [ ]:
import laspy
import pandas as pd
import pygmt

The data are from a recent LiDAR survey of Wellington, New Zealand that's published under a [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/) license.
- OpenTopography link: https://doi.org/10.5069/G9K935QX
- Bulk download location: https://opentopography.s3.sdsc.edu/minio/pc-bulk/NZ19_Wellington
- Official 1m DSM from LINZ: https://data.linz.govt.nz/layer/105024-wellington-city-lidar-1m-dsm-2019-2020

References:
- https://medium.com/on-location/from-points-to-pixels-creating-digital-elevation-models-from-open-topography-point-clouds-abe616d06860
- https://github.com/GenericMappingTools/foss4g2019oceania/blob/v1/3_lidar_to_surface.ipynb
- https://github.com/weiji14/30DayMapChallenge2021/blob/v0.3.1/day17_land.py

We can use `pygmt.which` to download a remote file to our current working directory

In [ ]:
lazfile = pygmt.which(
    fname="https://opentopography.s3.sdsc.edu/pc-bulk/NZ19_Wellington/CL2_BQ31_2019_1000_2138.laz",
    download=True,
)

For convenience, we'll store the data as a pandas DataFrame and filter out noisy data

In [ ]:
lazdata = laspy.read(source=lazfile)
df = pd.DataFrame(
    data={
        "x": lazdata.x.scaled_array(),
        "y": lazdata.y.scaled_array(),
        "z": lazdata.z.scaled_array(),
        "classification": lazdata.classification,
    }
)
df = df.query(expr="classification != 18")

We can use `pygmt.info` to get the bounding box for the data

In [ ]:
region = pygmt.info(data=df[["x", "y"]], spacing=1)

We'll preprocess the data by using `blockmedian` to get the 99th quantile of data for each 1m block inside the bounding box

In [ ]:
df_trimmed = pygmt.blockmedian(
    data=df[["x", "y", "z"]], T=0.99, spacing="1+e", region=region, C=True
)
df_trimmed

We'll create a Digital Surface Elevation Model with a spatial resolution of 1m using adjustable tension splines via `pygmt.surface`

In [ ]:
grid = pygmt.surface(
    x=df_trimmed.x,
    y=df_trimmed.y,
    z=df_trimmed.z,
    spacing="1+e",
    region=region,
    T=0.35,  # tension factor
)
grid

Plot the Digital Surface Model (DSM) by passing the
[xarray.DataArray](https://docs.xarray.dev/en/v2022.03.0/generated/xarray.DataArray.html)
grid into
[pygmt.Figure.grdview](https://www.pygmt.org/v0.7.0/api/generated/pygmt.Figure.grdview).

In [ ]:
fig = pygmt.Figure()
fig.grdview(
    grid=grid,
    cmap="SCM/bukavu",
    surftype="s",  # surface plot
    perspective=[315, 30],  # azimuth bearing, and elevation angle
    zscale=0.02,  # vertical exaggeration
    shading=True,  # hillshading
    region=region,
    frame=[
        "xaf+lEasting",
        "yaf+lNorthing",
        "zaf+lElevation (m)",
        "+tOriental Bay, Wellington",
    ],
)
fig.show()

### Perform grid histogram equalization

In this example, we'll explore a simplified version of the PyGMT tutorial on [transforming grids based on a cumulative distribution function](https://www.pygmt.org/latest/tutorials/advanced/grid_equalization.html#sphx-glr-tutorials-advanced-grid-equalization-py).

Again, we'll first define our region of interest

In [ ]:
region = [-119.825, -119.4, 37.6, 37.825]

This time, we'll use a higher resolution version of the SRTM15+ earth relief dataset (Tozer et al., 2019)

In [ ]:
grid = pygmt.datasets.load_earth_relief(
    resolution="03s", region=region, registration="gridline"
)
# Store elevation values as a pandas DataFrame
grid_dist = pygmt.grd2xyz(grid=grid, output_type="pandas")["elevation"]

We can add some more customization to our figures using `pygmt.config`

In [ ]:
fig = pygmt.Figure()
pygmt.config(FORMAT_GEO_MAP="ddd.x", MAP_FRAME_TYPE="plain")
# Define the colormap for the figure
pygmt.makecpt(series=[500, 3540], cmap="turku")

We'll use the `subplot` and `set_panel` methods of `pygmt.Figure` to plot the DEM and the distribution of elevation values side-by-side

In [ ]:
# Setup subplots with two panels
with fig.subplot(
    nrows=1, ncols=2, figsize=("13.5c", "4c"), title="Digital Elevation Model"
):
    # Plot the original digital elevation model in the first panel
    with fig.set_panel(panel=0):
        fig.grdimage(grid=grid, region=region, projection="M?", frame="WSne", cmap=True)
    # Plot a histogram showing the z-value distribution in the original digital
    # elevation model
    with fig.set_panel(panel=1):
        fig.histogram(
            data=grid_dist,
            projection="X?",
            region=[500, 3600, 0, 20],
            series=[500, 3600, 100],
            frame=["wnSE", "xaf+lElevation (m)", "yaf+lPercent frequency"],
            cmap=True,
            histtype=1,
            pen="1p,black",
        )
        fig.colorbar(position="JMR+o1.5c/0c+w3c/0.3c", frame=True)
fig.show()

Create a new grid with the z-values representing the position of the original z-values in a given cumulative distribution.

In [ ]:
divisions = 10
linear = pygmt.grdhisteq.equalize_grid(grid=grid, divisions=divisions)
linear_dist = pygmt.grd2xyz(grid=linear, output_type="pandas")["z"]

Create a map showing the grid with a linear distribution of ten divisions along with a histogram of the data values.

In [ ]:
# Create an instance of the Figure class
fig = pygmt.Figure()
# Define figure configuration
pygmt.config(FORMAT_GEO_MAP="ddd.x", MAP_FRAME_TYPE="plain")
# Define the colormap for the figure
pygmt.makecpt(series=[0, divisions, 1], cmap="lajolla")
# Setup subplots with two panels
with fig.subplot(
    nrows=1, ncols=2, figsize=("13.5c", "4c"), title="Linear distribution"
):
    # Plot the grid with a linear distribution in the first panel
    with fig.set_panel(panel=0):
        fig.grdimage(
            grid=linear, region=region, projection="M?", frame="WSne", cmap=True
        )
    # Plot a histogram showing the linear z-value distribution
    with fig.set_panel(panel=1):
        fig.histogram(
            data=linear_dist,
            projection="X?",
            region=[-1, divisions, 0, 40],
            series=[0, divisions, 1],
            frame=["wnSE", "xaf+lDivision", "yaf+lPercent frequency"],
            cmap=True,
            histtype=1,
            pen="1p,black",
            center=True,
        )
        fig.colorbar(position="JMR+o1.5c/0c+w3c/0.3c", frame=True)
fig.show()

### Find out more

For more data visualization and processing examples, check out our [tutorials](https://www.pygmt.org/latest/tutorials/index.html), [gallery examples](https://www.pygmt.org/latest/gallery/index.html), and the [external resources](https://www.pygmt.org/latest/external_resources.html)!